In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install fasttext
!pip install collections
!pip install tqdm
!pip install barbar
# !pip uninstall emoji
!pip install emoji==1.7
!pip install emojis
!pip install --upgrade urllib3
!pip install brokenaxes

ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections


In [3]:
!pip install GPUtil

In [4]:
!pip install transformers

In [5]:
!pip install pytorch_pretrained_bert

In [6]:
!mkdir ckpts
!mkdir reports
!mkdir results

mkdir: cannot create directory ‘ckpts’: File exists
mkdir: cannot create directory ‘reports’: File exists
mkdir: cannot create directory ‘results’: File exists


In [7]:
import gensim
from tqdm import tqdm
from collections import defaultdict
import fasttext
import fasttext.util
# from gensim.models.utils_any2vec import ft_ngram_hashes  # This function is used to calculate hashes from ngrams to determine position in ngram matrix
from gensim.models.fasttext import ft_ngram_hashes

import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from brokenaxes import brokenaxes
from numpy.random import RandomState
import operator
from scipy.interpolate import make_interp_spline
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import json

from sklearn.metrics import accuracy_score, f1_score, classification_report
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
import argparse
import numpy as np
import pandas as pd
import random
from transformers import AdamW, get_linear_schedule_with_warmup
from barbar import Bar
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import os
import re
from emoji import UNICODE_EMOJI
import sys
import emojis
import numpy
import shutil

from statistics import mean
import math
from torch.autograd import Variable
%matplotlib inline
import gensim
import re
import numpy as np
from nltk import ngrams
from emoji import UNICODE_EMOJI
from nltk.tokenize import word_tokenize
from collections import defaultdict
from tqdm import tqdm_notebook
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
folder_path = 'Tuning'

In [9]:
results_h = f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/results_h.jsonl'
results_h_2 = f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/results_h_2.jsonl'
results_t = f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/results_t.jsonl'
results_t_2 = f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/results_t_2.jsonl'

In [10]:
ft = gensim.models.KeyedVectors.load("/content/drive/MyDrive/iSarcasm/fasttext/ft_250.model")


In [11]:
model_paths_base = []
model_paths_cls = []
base_dict  = {}
cls_dict={}
directory = f'drive/MyDrive/iSarcasm/{folder_path}/mix/models'

for filename in os.listdir(directory):
    arr = filename.split("_")
    if filename.startswith('best_basemodel_')  :
        # shutil.copy(f"{directory}/{filename}", f"ckpts/{filename}")
        # best_basemodel_sarcasm_16_FL_1e-05_ml70_1_16826_sarcat_alpha_e_1_1e-05_16.pth
        model_name = f"{arr[3]}_{arr[4]}_{arr[5]}_{arr[7]}_{arr[8]}_{arr[10]}_{arr[11]}_{arr[12]}_{arr[13]}_{arr[14]}"
        base_dict[model_name]= filename
        model_paths_base.append(filename)
    if filename.startswith('best_swe_marbert_sarcasm_') :
        # shutil.copy(f"{directory}/{filename}", f"ckpts/{filename}")
        # best_swe_marbert_sarcasm_16_FL_1e-05_ml70_3_16826_sarcat_alpha_e_1_1e-05_16.pth
        model_name = f"{arr[4]}_{arr[5]}_{arr[6]}_{arr[8]}_{arr[9]}_{arr[11]}_{arr[12]}_{arr[13]}_{arr[14]}_{arr[15]}"
        cls_dict[model_name]=filename
        model_paths_cls.append(filename)


In [12]:
model_paths_cls

['best_swe_marbert_sarcasm_16_FL_1e-05_ml70_1_16826_sarcat_alpha_e_1_1e-05_16.pth',
 'best_swe_marbert_sarcasm_16_FL_1e-05_ml70_1_16826_sarcat_alpha_e_2_1e-05_16.pth',
 'best_swe_marbert_sarcasm_16_FL_1e-05_ml70_1_16826_sarcat_alpha_e_3_1e-05_16.pth',
 'best_swe_marbert_sarcasm_16_FL_3e-05_ml70_1_16826_sarcat_alpha_e_1_3e-05_16.pth',
 'best_swe_marbert_sarcasm_16_FL_2e-05_ml70_1_16826_sarcat_alpha_e_1_2e-05_16.pth',
 'best_swe_marbert_sarcasm_16_FL_3e-05_ml70_1_16826_sarcat_alpha_e_2_3e-05_16.pth',
 'best_swe_marbert_sarcasm_16_FL_2e-05_ml70_1_16826_sarcat_alpha_e_2_2e-05_16.pth',
 'best_swe_marbert_sarcasm_16_FL_3e-05_ml70_1_16826_sarcat_alpha_e_3_3e-05_16.pth',
 'best_swe_marbert_sarcasm_16_FL_2e-05_ml70_1_16826_sarcat_alpha_e_3_2e-05_16.pth',
 'best_swe_marbert_sarcasm_16_FL_4e-05_ml70_1_16826_sarcat_alpha_e_1_4e-05_16.pth',
 'best_swe_marbert_sarcasm_16_FL_4e-05_ml70_1_16826_sarcat_alpha_e_2_4e-05_16.pth',
 'best_swe_marbert_sarcasm_32_FL_1e-05_ml70_1_16826_sarcat_alpha_e_1_1e-05_3

In [13]:
len(model_paths_cls)

25

In [14]:
print(model_paths_base,"\n",model_paths_cls,"\n",base_dict,"\n",cls_dict)
len(base_dict)

['best_basemodel_sarcasm_16_FL_1e-05_ml70_1_16826_sarcat_alpha_e_1_1e-05_16.pth', 'best_basemodel_sarcasm_16_FL_1e-05_ml70_1_16826_sarcat_alpha_e_2_1e-05_16.pth', 'best_basemodel_sarcasm_16_FL_1e-05_ml70_1_16826_sarcat_alpha_e_3_1e-05_16.pth', 'best_basemodel_sarcasm_16_FL_3e-05_ml70_1_16826_sarcat_alpha_e_1_3e-05_16.pth', 'best_basemodel_sarcasm_16_FL_2e-05_ml70_1_16826_sarcat_alpha_e_1_2e-05_16.pth', 'best_basemodel_sarcasm_16_FL_3e-05_ml70_1_16826_sarcat_alpha_e_2_3e-05_16.pth', 'best_basemodel_sarcasm_16_FL_2e-05_ml70_1_16826_sarcat_alpha_e_2_2e-05_16.pth', 'best_basemodel_sarcasm_16_FL_3e-05_ml70_1_16826_sarcat_alpha_e_3_3e-05_16.pth', 'best_basemodel_sarcasm_16_FL_2e-05_ml70_1_16826_sarcat_alpha_e_3_2e-05_16.pth', 'best_basemodel_sarcasm_16_FL_4e-05_ml70_1_16826_sarcat_alpha_e_1_4e-05_16.pth', 'best_basemodel_sarcasm_16_FL_4e-05_ml70_1_16826_sarcat_alpha_e_2_4e-05_16.pth', 'best_basemodel_sarcasm_32_FL_1e-05_ml70_1_16826_sarcat_alpha_e_1_1e-05_32.pth', 'best_basemodel_sarcasm_16_

25

In [15]:
vector_size = 300

In [16]:
class FocalLoss_Ori(nn.Module):
    """
    This is a implementation of Focal Loss with smooth label cross entropy supported which is proposed in
    'Focal Loss for Dense Object Detection. (https://arxiv.org/abs/1708.02002)'
    Focal_Loss= -1*alpha*((1-pt)**gamma)*log(pt)
    Args:
        num_class: number of classes
        alpha: class balance factor
        gamma:
        ignore_index:
        reduction:
    """

    def __init__(self, num_class, alpha=None, gamma=2, ignore_index=None, reduction='mean'):
        super(FocalLoss_Ori, self).__init__()
        self.num_class = num_class
        self.gamma = gamma
        self.reduction = reduction
        self.smooth = 1e-4
        self.ignore_index = ignore_index
        self.alpha = alpha
        if alpha is None:
            self.alpha = torch.ones(num_class, )
        elif isinstance(alpha, (int, float)):
            self.alpha = torch.as_tensor([alpha] * num_class)
        elif isinstance(alpha, (list, np.ndarray)):
            self.alpha = torch.as_tensor(alpha)
        if self.alpha.shape[0] != num_class:
            raise RuntimeError('the length not equal to number of class')

    def forward(self, logit, target):
        # assert isinstance(self.alpha,torch.Tensor)\
        N, C = logit.shape[:2]
        alpha = self.alpha.to(logit.device)
        prob = F.softmax(logit, dim=1)
        if prob.dim() > 2:
            # N,C,d1,d2 -> N,C,m (m=d1*d2*...)
            prob = prob.view(N, C, -1)
            prob = prob.transpose(1, 2).contiguous()  # [N,C,d1*d2..] -> [N,d1*d2..,C]
            prob = prob.view(-1, prob.size(-1))  # [N,d1*d2..,C]-> [N*d1*d2..,C]
        ori_shp = target.shape
        target = target.view(-1, 1)  # [N,d1,d2,...]->[N*d1*d2*...,1]
        valid_mask = None
        if self.ignore_index is not None:
            valid_mask = target != self.ignore_index
            target = target * valid_mask

        # ----------memory saving way--------
        prob = prob.gather(1, target).view(-1) + self.smooth  # avoid nan
        logpt = torch.log(prob)
        # alpha_class = alpha.gather(0, target.view(-1))
        alpha_class = alpha[target.squeeze().long()]
        class_weight = -alpha_class * torch.pow(torch.sub(1.0, prob), self.gamma)
        loss = class_weight * logpt
        if valid_mask is not None:
            loss = loss * valid_mask.squeeze()

        if self.reduction == 'mean':
            loss = loss.mean()
            if valid_mask is not None:
                loss = loss.sum() / valid_mask.sum()

        elif self.reduction == 'none':
            loss = loss.view(ori_shp)
        return loss





In [17]:
def get_vec_word(word1):
      flag = False
      try:
          idx = ft.key_to_index[word]
          vector2= ft.vectors[idx]
          return vector2
      except:
          tmp_v = np.random.uniform(-0.25, 0.25, 300)
          c = 0
          new_vector = np.zeros(vector_size, dtype=np.float32)
          char_ngrams = []
          word = '<' + word1 +'>'
          for n in range(5, 5 + 1):
              char_ngrams = [word[i:i + n] for i in range(len(word) - n + 1)]
              char_ngrams.append(word1)
              for x in char_ngrams:
                  try:
                    idx = ft.key_to_index[x]
                    vector2= ft.vectors[idx]
                    new_vector += vector2
                    c+=1
                  except:
                    pass
              all_zeros = not new_vector.any()
              if(all_zeros):
                new_vector = tmp_v
              else:
                new_vector/=c
              return new_vector

# split_data


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:

class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, bias=True):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias

        self.xh = nn.Linear(input_size, hidden_size * 4, bias=bias)
        self.hh = nn.Linear(hidden_size, hidden_size * 4, bias=bias)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / np.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)

    def forward(self, input, hx=None):

        # Inputs:
        #       input: of shape (batch_size, input_size)
        #       hx: of shape (batch_size, hidden_size)
        # Outputs:
        #       hy: of shape (batch_size, hidden_size)
        #       cy: of shape (batch_size, hidden_size)

        if hx is None:
            hx = Variable(input.new_zeros(input.size(0), self.hidden_size))
            hx = (hx, hx)

        hx, cx = hx

        gates = self.xh(input) + self.hh(hx)

        # Get gates (i_t, f_t, g_t, o_t)
        input_gate, forget_gate, cell_gate, output_gate = gates.chunk(4, 1)

        i_t = torch.sigmoid(input_gate)
        f_t = torch.sigmoid(forget_gate)
        g_t = torch.tanh(cell_gate)
        o_t = torch.sigmoid(output_gate)

        cy = cx * f_t + i_t * g_t

        hy = o_t * torch.tanh(cy)


        return (hy, cy)


class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, bias, output_size):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bias = bias
        self.output_size = output_size

        self.rnn_cell_list = nn.ModuleList()

        self.rnn_cell_list.append(LSTMCell(self.input_size,
                                            self.hidden_size,
                                            self.bias))
        for l in range(1, self.num_layers):
            self.rnn_cell_list.append(LSTMCell(self.hidden_size,
                                                self.hidden_size,
                                                self.bias))

        self.fc = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hx=None):

        # Input of shape (batch_size, seqence length , input_size)
        #
        # Output of shape (batch_size, output_size)

        if hx is None:
            if torch.cuda.is_available():
                h0 = Variable(torch.zeros(self.num_layers, input.size(0), self.hidden_size).cuda())
            else:
                h0 = Variable(torch.zeros(self.num_layers, input.size(0), self.hidden_size))
        else:
             h0 = hx

        outs = []

        hidden = list()
        for layer in range(self.num_layers):
            hidden.append((h0[layer, :, :], h0[layer, :, :]))

        for t in range(input.size(1)):

            for layer in range(self.num_layers):

                if layer == 0:
                    hidden_l = self.rnn_cell_list[layer](
                        input[:, t, :],
                        (hidden[layer][0],hidden[layer][1])
                        )
                else:
                    hidden_l = self.rnn_cell_list[layer](
                        hidden[layer - 1][0],
                        (hidden[layer][0], hidden[layer][1])
                        )

                hidden[layer] = hidden_l

            outs.append(hidden_l[0])

        out = outs[-1].squeeze()

        out = self.fc(out)

        return out

In [20]:
def split_data(data_name):
      if(data_name == 'ArSarcasm'):
        df  = pd.read_csv('/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm/ArSarcasm_train.csv')
        folder = '/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm/split/'
        df['sarcasm'] = df['sarcasm'].map({True: 1, False: 0})
        df.rename(columns = {'sarcasm':'sarcastic'}, inplace = True)

      elif(data_name == 'ArSarcasm_v2'):
        df  = pd.read_csv('/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm_v2/training_data.csv')
        folder = '/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm_v2/split/'
        df['sarcasm'] = df['sarcasm'].map({True: 1, False: 0})
        df.rename(columns = {'sarcasm':'sarcastic'}, inplace = True)

      rng = RandomState()
      data = df.sample(frac=1).reset_index(drop=True)
      train, test = train_test_split(data, test_size=0.2, stratify=data["sarcastic"].values, random_state=42, shuffle=True)#

      train.to_csv('train.csv')
      test.to_csv('val.csv')

      shutil.copy('train.csv', f"{folder}train.csv")
      shutil.copy('val.csv', f"{folder}val.csv")


In [21]:
def set_seed(seed_value=42):
    """Set seed for reproducibility."""

    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

In [22]:
def build_vocab(t_model):

  word2idx = {}
  # Add <pad> and <unk> tokens to the vocabulary
  word2idx['<pad>'] = 0
  word2idx['<unk>'] = 1

  d =t_model.vector_size

  embeddings = np.random.uniform(-0.25, 0.25, (len(t_model.key_to_index)+2, d))

  embeddings[word2idx['<pad>']] = np.zeros((d,))
  idx = 2
  for word in t_model.key_to_index:
      if word not in word2idx:
            word2idx[word] = idx
            embeddings[word2idx[word]] = np.array(get_vec_word(word), dtype=np.float32)
            idx += 1

  print(len(embeddings), len(word2idx))
  return  word2idx, embeddings

In [23]:
class TrainDataset(Dataset):
    def __init__(self, df, static_input_ids2, pretraine_path='xlm-roberta-base', max_length=128):
        self.df = df
        self.max_length = max_length
        self.static_input_ids2= static_input_ids2
        self.tokenizer = AutoTokenizer.from_pretrained(pretraine_path)

    def __getitem__(self, index):
        text = self.df.iloc[index]['tweet']
        sarc_label = self.df.iloc[index]["sarcastic"]
        sent_label = self.df.iloc[index]["sentiment"]
        static_input_ids2 = self.static_input_ids2[index]
        encoded_input = self.tokenizer(
                text,
                max_length = self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors="pt",
            )

        input_ids = encoded_input["input_ids"]
        attention_mask = encoded_input["attention_mask"] if "attention_mask" in encoded_input else None

        data_input = {
            "input_ids":input_ids.flatten(),
            "attention_mask": attention_mask.flatten()
        }

        sarc_label_input ={
            "sarcasm": torch.tensor(sarc_label, dtype=torch.float),
        }
        sent_label_input ={
            "sentiment": torch.tensor(sent_label, dtype=torch.float),
        }
        static_input_ids ={
            "static_input_ids":torch.from_numpy(np.asarray(static_input_ids2)),
        }

        return data_input, sarc_label_input, sent_label_input, static_input_ids

    def __len__(self):
        return self.df.shape[0]

In [24]:
def label_rule(mis_preds, cat_preds):
    for i in range(len(mis_preds)):
        if mis_preds[i] == 0:
            cat_preds[i] = 0
    return cat_preds


def accuracy(preds, y):
    all_output = preds.float().cpu()
    all_label = y.float().cpu()
    _, predict = torch.max(all_output, 1)
    acc = accuracy_score(all_label.numpy(), torch.squeeze(predict).float().numpy())
    return acc

def calc_accuracy(preds,y):
    predict = torch.argmax(preds, dim=1)
    accuracy = torch.sum(predict == y.squeeze()).float().item()
    return accuracy / float(preds.size()[0])

def calc_f1_sarcasm(preds,y):

    predict = torch.argmax(preds, dim=1)
    f1_sarcastic = f1_score(y, predict, average='binary', pos_label=1)
    return f1_sarcastic

def calc_f1_score(preds,y):
    predict = torch.argmax(preds, dim=1)
    f1_score_macro = f1_score(y, predict, average='macro')
    return f1_score_macro

def binary_accuracy2(preds, y):
    # round predictions to the closest integer
    rounded_preds = torch.round(preds).squeeze()

    correct = (rounded_preds == y).float()
    acc = correct.sum() / (y.size(0))
    return acc

def binary_accuracy(preds, y):
    # round predictions to the closest integer
    rounded_preds = torch.round(preds).squeeze()

    correct = (rounded_preds == y).float()
    acc = correct.sum() / (y.size(0) * y.size(1))
    return acc

In [25]:
folder_dic = {
    "ArSarcasm": '/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm',
    "ArSarcasm_v2": '/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm_v2'
}

In [26]:
folder_dic['ArSarcasm']

'/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm'

In [27]:
dic= {
      "egypt": 'المصرية',
	  "nile": 'المصرية',
	  "msa": "اللغة العربية الفصحى",
	  "magreb": "المغربية",
	  "gulf": "الخليجية",
	  "levant": "الشامية"
}


In [28]:
def is_emoji(s):
    return s in UNICODE_EMOJI

# add space near your emoji
def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

def preprocess(text):
    sent = add_space(text)
    sent = re.sub(r'(?:@[\w_]+)', "user", sent)
    sent = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', "url", sent)
    sent = sent.replace('_', ' ')
    sent = sent.replace('#', ' ')
    return sent

def prepare_text(df, col):
    if col == 'tweet':
        df['dialect'] = df['dialect'].map(dic)
    for i in range(df.shape[0]):
        df.loc[i, col] = df.loc[i, 'dialect'] + ' [SEP] ' + df.loc[i, col]


    return df


In [29]:
def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1 or classname.find('ConvTranspose2d') != -1:
        nn.init.kaiming_uniform_(m.weight)
        nn.init.zeros_(m.bias)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.zeros_(m.bias)
    elif classname.find('Linear') != -1:
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)




class TransformerLayer(nn.Module):
    def __init__(self,both=True,
                pretrained_path='aubmindlab/bert-base-arabert'):
        super(TransformerLayer, self).__init__()

        self.both = both
        self.transformer = AutoModel.from_pretrained(pretrained_path, output_hidden_states=True)


    def forward(self, input_ids=None, attention_mask=None):
        outputs = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        # (output_last_layer, pooled_cls, (output_layers))
        # output[0] (8, seqlen=64, 768) cls [8, 768] ( 12 (8, seqlen=64, 768))

        return outputs

    def output_num(self):
        return self.transformer.config.hidden_size




In [30]:
def evaluate(base_model, mt_classifier, iterator, criterion):
    # initialize every epoch
    acc_sarcasm= 0
    f1_sarcasm =0
    f1_score_macro = 0
    loss= 0

    #all_sarcasm_outputs = []
    all_sarcasm_outputs = np.array([])
    all_sarcasm_labels = np.array([])


    # set the model in eval phase
    base_model.eval()
    mt_classifier.eval()
    with torch.no_grad():
        for data_input, sarc_label_input, sent_label_input, static_input_ids  in Bar(iterator):

            for k, v in data_input.items():
                data_input[k] = v.to(device)

            for k, v in sarc_label_input.items():
                sarc_label_input[k] = v.long().to(device)

            for k, v in sent_label_input.items():
                sent_label_input[k] = v.long().to(device)

            for k, v in static_input_ids.items():
                # print(k,v)
                static_input_ids[k] = v.to(device)


            sarcasm_target = sarc_label_input['sarcasm']

            # forward pass
            s_input_ids = static_input_ids['static_input_ids']
            output = base_model(**data_input)
            sarcasm_logits = mt_classifier(s_input_ids, output)
            # logits = sarcasm_logits[:,:2]

            sarcasm_probs = torch.softmax(sarcasm_logits, dim=1)

            # compute the loss
            loss_ = criterion(sarcasm_logits, sarcasm_target)

            # compute the running accuracy and losses
            acc_sarcasm += calc_accuracy(sarcasm_probs, sarcasm_target)
            # loss_sarc += loss_sarcasm.item()

            _, predicted_sarcasm = torch.max(sarcasm_probs, 1)


            #all_sarcasm_outputs.extend(predicted_sarcasm.squeeze().int().cpu().numpy().tolist())
            all_sarcasm_outputs = np.append(all_sarcasm_outputs, predicted_sarcasm.squeeze().cpu().numpy())
            #all_sarcasm_labels.extend(sarcasm_target.squeeze().int().cpu().numpy().tolist())
            all_sarcasm_labels = np.append(all_sarcasm_labels, sarcasm_target.squeeze().cpu().numpy())


    all_sarcasm_outputs = all_sarcasm_outputs.reshape(-1)
    all_sarcasm_labels = all_sarcasm_labels.reshape(-1)

    fscore_macro_sarc = f1_score(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs, average='macro')
    fscore_sarcasm = f1_score(all_sarcasm_labels, all_sarcasm_outputs, average='binary', pos_label=1)

    report_sarcasm = classification_report(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs,digits=4)


    # accuracies = { 'accuracy': acc_sarcasm / len(iterator), 'f1_sarcastic': fscore_sarcasm, "f1_score": fscore_macro, 'report_sarcasm': report_sarcasm}
    accuracies = { 'acc_sarc': acc_sarcasm / len(iterator), 'f1_sarcastic': fscore_sarcasm, "f1_score_sarc": fscore_macro_sarc}
    losses = { 'loss': loss_ / len(iterator)}

    return accuracies, losses


In [31]:
def predict(base_model, mt_classifier, iterator):
    # initialize every epoch
    acc_sarcasm= 0
    loss= 0

    f1_sarcasm =0
    f1_score_macro = 0

    #all_sarcasm_outputs = []
    all_sarcasm_outputs = np.array([])
    all_sarcasm_labels = np.array([])

    # set the model in eval phase
    base_model.eval()
    mt_classifier.eval()

    with torch.no_grad():
      for data_input, sarc_label_input, sent_label_input, static_input_ids  in Bar(iterator):

            for k, v in data_input.items():
                data_input[k] = v.to(device)

            for k, v in sarc_label_input.items():
                sarc_label_input[k] = v.long().to(device)

            for k, v in sent_label_input.items():
                sent_label_input[k] = v.long().to(device)
            for k, v in static_input_ids.items():
                static_input_ids[k] = v.to(device)



            # forward pass
            sarcasm_target = sarc_label_input['sarcasm']

            output = base_model(**data_input)
            s_input_ids = static_input_ids['static_input_ids']
            sarcasm_logits = mt_classifier(s_input_ids, output)

            # logits = sarcasm_logits[:,:2]
            sarcasm_probs = torch.softmax(sarcasm_logits, dim=1)

            # compute the loss
            acc_sarcasm += calc_accuracy(sarcasm_probs, sarcasm_target)
            _, predicted_sarcasm = torch.max(sarcasm_probs, 1)


            #all_sarcasm_outputs.extend(predicted_sarcasm.squeeze().int().cpu().numpy().tolist())
            all_sarcasm_outputs = np.append(all_sarcasm_outputs, predicted_sarcasm.squeeze().cpu().numpy())
            all_sarcasm_labels = np.append(all_sarcasm_labels, sarcasm_target.squeeze().cpu().numpy())

    all_sarcasm_outputs = all_sarcasm_outputs.reshape(-1)
    all_sarcasm_labels = all_sarcasm_labels.reshape(-1)

    fscore_macro_sarc = f1_score(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs, average='macro')
    fscore_sarcasm = f1_score(all_sarcasm_labels, all_sarcasm_outputs, average='binary', pos_label=1)


    report_sarcasm = classification_report(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs,digits=4)

    accuracies = { 'acc_sarc': acc_sarcasm / len(iterator), 'f1_sarcastic': fscore_sarcasm, "f1_score_sarc": fscore_macro_sarc}

    return accuracies, all_sarcasm_outputs, all_sarcasm_labels


In [32]:
dic_labels ={
    "NEG": 0,
    "NEU": 1,
    "POS": 2
}
def loadTrainValData2(data_name="ArSarcasm_v2",size=0.2, batchsize=16, num_worker=0, pretraine_path="marbert", seed=42, max_length=128):

    path = "/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm_v2/split/train.csv"
    data = pd.read_csv(path, encoding='utf-8')
    data = data[~data.tweet.isna()]
    data['tweet'] = data['tweet'].apply(lambda x: preprocess(x))
    data = prepare_text(data, col='tweet')
    for i in range (len(data['sentiment'])):
        data['sentiment'][i] = dic_labels[data['sentiment'][i]]

    data = data[["tweet", "sarcastic", "sentiment"]]
    data = data.sample(frac=1).reset_index(drop=True)
    # data.head(10)
    df_train = data
    df_train = df_train.sample(frac=1).reset_index(drop=True)


    path = "/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm_v2/split/val.csv"


    data = pd.read_csv(path, encoding='utf-8')
    data = data[~data.tweet.isna()]
    data['tweet'] = data['tweet'].apply(lambda x: preprocess(x))
    data = prepare_text(data, col='tweet')

    for i in range (len(data['sentiment'])):
        data['sentiment'][i] = dic_labels[data['sentiment'][i]]
    # data.head(10)
    df_test = data
    # DF_train = TrainDataset(df_train, pretraine_path, max_length)
    # DF_test = TrainDataset(df_test, pretraine_path, max_length)

    # DF_train_loader = DataLoader(dataset=DF_train, batch_size=batchsize, shuffle=True,
    #                              num_workers=num_worker)
    # DF_test_loader = DataLoader(dataset=DF_test, batch_size=batchsize, shuffle=False,
    #                             num_workers=num_worker)
    # return DF_train_loader, DF_test_loader
    return df_train, df_test

# len(embeddings)

def tokenize(texts):
    """Tokenize texts, build vocabulary and find maximum sentence length.

    Args:
        texts (List[str]): List of text data

    Returns:
        tokenized_texts (List[List[str]]): List of list of tokens
        word2idx (Dict): Vocabulary built from the corpus
        max_len (int): Maximum sentence length
    """
    tokenized_texts = []
    for sent in texts:
        tokenized_sent = word_tokenize(sent)
        # Add `tokenized_sent` to `tokenized_texts`
        tokenized_texts.append(tokenized_sent)
    return tokenized_texts
def encode(tokenized_texts=None, word2idx=None, max_len=None):
    """Pad each sentence to the maximum sentence length and encode tokens to
    their index in the vocabulary.

    Returns:
        input_ids (np.array): Array of token indexes in the vocabulary with
            shape (N, max_len). It will the input of our CNN model.
    """

    input_ids = []

    for tokenized_sent in tokenized_texts:
        # Pad sentences to max_len
        tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))
        # Encode tokens to input_ids
        input_id = []
        # input_id = [(if token in word2idx: word2idx.get(token) else: input_id = word2idx.get('<unk>')) for token in tokenized_sent]
        for token in tokenized_sent:
            if token in word2idx:
              input_id_i = word2idx.get(token)
            else:
              input_id_i = word2idx.get('<unk>')
            input_id.append(input_id_i)

        input_ids.append(input_id)


    return input_ids



In [33]:
class ATTClassifier(nn.Module):
    def __init__(self, in_feature, class_num_sarc=1, class_num_sent=3, dropout_prob=0.2):
        super(ATTClassifier, self).__init__()

        self.fc11 = nn.Linear(in_feature, 512)
        self.do1 = nn.Dropout(dropout_prob)
        self.r1= nn.ReLU()
        self.fc12  = nn.Linear(512, class_num_sarc)

        self.fc21 = nn.Linear(in_feature, 512)
        self.do2 = nn.Dropout(dropout_prob)
        self.r2= nn.ReLU()
        self.fc22= nn.Linear(512, class_num_sent)

        self.apply(init_weights)

    def forward(self, x):

        xx = x[1]
        out_sarc = self.fc12( self.r1 ( self.do1( self.fc11 (xx))))
        out_sent = self.fc22( self.r2 ( self.do2( self.fc21 (xx))))
        return [out_sarc,out_sent]

In [34]:
class SWE_MARBERT(nn.Module):
    def __init__(self,
                 in_feature,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 num_classes=2,
                 dropout=0.5,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100]):



        super(SWE_MARBERT, self).__init__()
        # Embedding layer

        self.vocab_size, self.embed_dim = pretrained_embedding.shape
        self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)


        self.fc11 = nn.Linear(in_feature, 512)
        self.fc21 = nn.Linear(in_feature, 512)

        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # self.lstm_model = LSTM(input_size=self.embed_dim, hidden_size=self.embed_dim, num_layers=1, bias=True , output_size=self.embed_dim)

        self.lstm_model = LSTM(input_size=np.sum(num_filters), hidden_size=np.sum(num_filters), num_layers=1, bias=True , output_size=np.sum(num_filters))

        # self.fc = nn.Linear( self.embed_dim + 2*512 , num_classes)
        self.fc = nn.Linear( np.sum(num_filters) + 2*512 , num_classes)



        self.dropout = nn.Dropout(p=dropout)
        # self.avg_pool = nn.AvgPool1d(3)

    def forward(self, input_ids, x):
        """Perform a forward pass through the network.

        Args:
            input_ids (torch.Tensor): A tensor of token ids with shape
                (batch_size, max_sent_length)

        Returns:
            logits (torch.Tensor): Output logits with shape (batch_size,
                n_classes)
        """
        # print("input_ids", input_ids)
        # print("type input_ids", type(input_ids))
        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_reshaped = self.embedding(input_ids).float()
        #  Output shape: (b, embed_dim, max_len)
        x_reshaped = x_reshaped.permute(0, 2, 1)
        # #reshape it to(b, embed_dim)
        # x_reshaped = torch.mean(x_embed, dim=1)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]

        # # Concatenate x_pool_list to feed the fully connected layer.
        # # Output shape: (b, sum(num_filters))
        x_pool = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        x_pool  = x_pool.reshape([x_pool.shape[0], 1, x_pool.shape[1]])
        x_lstm = self.lstm_model(x_pool)

        # (output_last_layer, pooled_cls, (output_layers))
        # output[0] (b=8, seqlen=64, 768) cls [8, 768] ( 12 (8, seqlen=64, 768))
        # print((x.keys()))
        xx = x[1]
        out_sarc = self.fc11 (xx)
        out_sent = self.fc21 (xx)
        x_fc = torch.cat([out_sarc, out_sent, x_lstm], dim=1)
        # x_fc = torch.cat([out_sarc, out_sent, x_pool], dim=1)

        # cls [b=8, 768]
        # x_fc = torch.cat([xx, xx, x_lstm], dim=1)
        # Compute logits. Output shape: (b, n_classes)
        # logits = self.fc(self.dropout(x_fc))
        logits = self.fc(self.dropout(x_fc))

        return logits


In [35]:
def loadTestData(data_name, batchsize=16, num_worker=2, pretraine_path="xlm-roberta-base", max_length=128):

    path = "/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm_v2/split/test.csv"

    data = pd.read_csv(path, encoding='utf-8')
    data['tweet'] = data['tweet'].apply(lambda x:preprocess(x))
    data = prepare_text(data, col='tweet')

    for i in range (len(data['sentiment'])):
        data['sentiment'][i] = dic_labels[data['sentiment'][i]]

    data_df = data[["tweet", "sarcastic", "sentiment"]]
    # DF_test = TrainDataset(data, pretraine_path, max_length)

    # DF_test_loader = DataLoader(dataset=DF_test, batch_size=batchsize, shuffle=False,
    #                             num_workers=num_worker)


    return data_df

In [36]:
extra =''

In [37]:
def truncate(f, n):
    return math.floor(f * 10 ** n) / 10 ** n

In [38]:
data_arr = [ "ArSarcasm_v2"]

# Soup

In [39]:
data_i = data_arr[0]

In [40]:

args = {"data_location": os.path.expanduser('./data'),
        # "model_location": os.path.expanduser('./ckpts'),
        "model_location": os.path.expanduser(f'drive/MyDrive/iSarcasm/{folder_path}/mix/models'),
        "download_models": False,
        "eval_individual_models":True,
        "uniform_soup": True,
        "greedy_soup": True,
        "plot": True,
        "batch_size": 8,
        "workers": 8,
        "pretrained_path": "UBC-NLP/MARBERT",
        'loss' : 'FL',
        'max_length': 70

}

In [41]:
class SWE_MARBERT(nn.Module):
    def __init__(self,
                 in_feature,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 num_classes=2,
                 dropout=0.5,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100]):



        super(SWE_MARBERT, self).__init__()
        # Embedding layer

        self.vocab_size, self.embed_dim = pretrained_embedding.shape
        self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)


        self.fc11 = nn.Linear(in_feature, 512)
        self.fc21 = nn.Linear(in_feature, 512)

        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # self.lstm_model = LSTM(input_size=self.embed_dim, hidden_size=self.embed_dim, num_layers=1, bias=True , output_size=self.embed_dim)

        self.lstm_model = LSTM(input_size=np.sum(num_filters), hidden_size=np.sum(num_filters), num_layers=1, bias=True , output_size=np.sum(num_filters))

        # self.fc = nn.Linear( self.embed_dim + 2*512 , num_classes)
        self.fc = nn.Linear( np.sum(num_filters) + 2*512 , num_classes)



        self.dropout = nn.Dropout(p=dropout)
        # self.avg_pool = nn.AvgPool1d(3)

    def forward(self, input_ids, x):
        """Perform a forward pass through the network.

        Args:
            input_ids (torch.Tensor): A tensor of token ids with shape
                (batch_size, max_sent_length)

        Returns:
            logits (torch.Tensor): Output logits with shape (batch_size,
                n_classes)
        """
        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_reshaped = self.embedding(input_ids).float()
        #  Output shape: (b, embed_dim, max_len)
        x_reshaped = x_reshaped.permute(0, 2, 1)
        # #reshape it to(b, embed_dim)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]

        # # Concatenate x_pool_list to feed the fully connected layer.
        # # Output shape: (b, sum(num_filters))
        x_pool = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        x_pool  = x_pool.reshape([x_pool.shape[0], 1, x_pool.shape[1]])
        x_lstm = self.lstm_model(x_pool)

        # (output_last_layer, pooled_cls, (output_layers))
        # output[0] (b=8, seqlen=64, 768) cls [8, 768] ( 12 (8, seqlen=64, 768))
        xx = x[1]
        out_sarc = self.fc11 (xx)
        out_sent = self.fc21 (xx)
        x_fc = torch.cat([out_sarc, out_sent, x_lstm], dim=1)
        logits = self.fc(self.dropout(x_fc))

        return logits


In [42]:
def eval_full_soup(config, loader1, base_model, classifier):
    accuracies, all_outputs_pred, all_outputs_label = predict(base_model, classifier, loader1)
    return accuracies, all_outputs_pred, all_outputs_label

def get_model_from_sd(state_dict_cls, base_model):
    # print("state_dict_cls",state_dict_cls.keys())
    # feature_dim = state_dict_cls['Classifier.3.weight'].shape[1]
    num_classes = 2
    classifier = SWE_MARBERT(base_model.output_num(),
                              pretrained_embedding=embeddings,
                              freeze_embedding=False,
                              dropout=0.5)

    classifier.parameters()

    for p in classifier.parameters():
        p.data = p.data.float()

    classifier.load_state_dict(state_dict_cls)
    classifier = classifier.cuda()
    devices = [x for x in range(torch.cuda.device_count())]
    return torch.nn.DataParallel(classifier,  device_ids=devices)

In [43]:
args2 = {}
args2['lr_mult'] = 1.0
args2['seed'] = 16826
args2['num_worker'] = 4
args2['lang'] = 'ar'
args2['phase'] = 'train'
args2['lm_pretrained'] = 'marbert'

args2['batch_size'] = 8
config = {}
config["max_length"] = 70
max_length= config["max_length"]
config['args'] = args2
config["output_for_test"] = True
config["class_num"] = 1

config['lr_mult'] = args2['lr_mult']
config['batch_size'] = args2['batch_size']
config['lm'] = args2['lm_pretrained']

lang = args2['lang']
dosegmentation = False
if args2['lm_pretrained'] == 'marbert':
    config['pretrained_path'] = "UBC-NLP/MARBERT"

RANDOM_SEED = config['args']['seed']

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [44]:
df_train, df_val = loadTrainValData2(data_i, size = 0.2 ,batchsize=args['batch_size'], num_worker=0, pretraine_path=config['pretrained_path'], max_length=config['max_length'])


Streaming output truncated to the last 5000 lines.
<ipython-input-32-ce184af881ba>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'][i] = dic_labels[data['sentiment'][i]]
<ipython-input-32-ce184af881ba>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'][i] = dic_labels[data['sentiment'][i]]
<ipython-input-32-ce184af881ba>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'][i] = dic_lab

In [45]:
# train_texts = np.array(df_train['tweet'] )
val_texts = np.array(df_val['tweet'] )
# train_labels = np.array(df_train['sarcastic'])
val_labels = np.array(df_val['sarcastic'])
word2idx, embeddings = build_vocab(ft)
embeddings = torch.tensor(embeddings)
# tokenized_train_texts = tokenize(train_texts)
tokenized_val_texts = tokenize(val_texts)

572931 572930


In [46]:
# train_input_ids = encode(tokenized_train_texts, word2idx, max_length)
val_input_ids = encode(tokenized_val_texts, word2idx,max_length )

In [47]:
pretraine_path= config['pretrained_path']


In [48]:
DF_val = TrainDataset(df_val, val_input_ids, pretraine_path, max_length)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [49]:
batchsize = args['batch_size']


In [50]:
val_loader = DataLoader(dataset=DF_val, batch_size=batchsize, shuffle=False,
                                num_workers=0)

In [51]:
df_test = loadTestData(data_i,batchsize=args['batch_size'], num_worker=0, pretraine_path=config['pretrained_path'],  max_length=config['max_length'])

test_texts = np.array(df_test['tweet'] )
test_labels = np.array(df_test['sarcastic'])

tokenized_test_texts = tokenize(test_texts)
test_input_ids = encode(tokenized_test_texts, word2idx, max_length)

DF_test = TrainDataset(df_test, test_input_ids, config['pretrained_path'], max_length)
test_loader = DataLoader(dataset=DF_test, batch_size=args['batch_size'], shuffle=False,
                                num_workers=0)
len(test_input_ids)
len(df_test)

Streaming output truncated to the last 5000 lines.
<ipython-input-35-446f52f76cbb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'][i] = dic_labels[data['sentiment'][i]]
<ipython-input-35-446f52f76cbb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'][i] = dic_labels[data['sentiment'][i]]
<ipython-input-35-446f52f76cbb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'][i] = dic_lab

3000

In [52]:
# val_loader = loadValData(data_i,batchsize=args['batch_size'], num_worker=0, pretraine_path=args['pretrained_path'], max_length=args['max_length'])

# test_loader = loadTestData(data_i,batchsize=args['batch_size'], num_worker=0, pretraine_path=args['pretrained_path'],  max_length=args['max_length'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [53]:
UNIFORM_SOUP_RESULTS_FILE_H = f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/uniform_soup_results_h.jsonl'
UNIFORM_SOUP_RESULTS_FILE_T = f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/uniform_soup_results_t.jsonl'

# GREEDY_SOUP_RESULTS_FILE_H = 'greedy_soup_results_h.jsonl'
# GREEDY_SOUP_RESULTS_FILE_T = 'greedy_soup_results_t.jsonl'

In [54]:
# TOTAL_MODELS =16

In [55]:
# if os.path.exists(UNIFORM_SOUP_RESULTS_FILE_H):
#     os.remove(UNIFORM_SOUP_RESULTS_FILE_H)

# if os.path.exists(UNIFORM_SOUP_RESULTS_FILE_T):
#     os.remove(UNIFORM_SOUP_RESULTS_FILE_T)

# # print("n", n+1)

# INDIVIDUAL_MODEL_RESULTS_FILE_H = f"drive/MyDrive/iSarcasm/{folder_path}/mix/results/results_h.jsonl"
# individual_model_db_h = pd.read_json(INDIVIDUAL_MODEL_RESULTS_FILE_H, lines=True)

# NUM_MODELS = len(individual_model_db_h)
# n = NUM_MODELS - 1

# all_models_name = ', '.join(individual_model_db_h['model_name'])
# individual_model_db_h
# # args['pretrained_path']
# base_model = TransformerLayer(pretrained_path=config['pretrained_path'], both=True).to(device)
# j=0
# uniform_soup_base_h = None
# uniform_soup_cls_h = None

# for _, row in individual_model_db_h.iterrows():
#     # 	16_FL_1e-05_1_16826_alpha_e_1_1e-05_16
#     x = row['model_name']
#     print("model name", x)
#     # y is the file name of the base
#     # arr = x.split("_")
#     var3 = base_dict[x+'.pth']
#     var4 = cls_dict[x+'.pth']
#     state_dict_base = torch.load(args['model_location']+f"/{var3}")
#     state_dict_cls = torch.load(args['model_location']+f"/{var4}")

#     if j == 0:
#         uniform_soup_base_h = {k : v * (1./NUM_MODELS) for k, v in state_dict_base.items()}
#         uniform_soup_cls_h = {k : v * (1./NUM_MODELS) for k, v in state_dict_cls.items()}
#         j = 1
#     else:
#         uniform_soup_base_h = {k : v * (1./NUM_MODELS) + uniform_soup_base_h[k] for k, v in state_dict_base.items()}
#         uniform_soup_cls_h = {k : v * (1./NUM_MODELS) + uniform_soup_cls_h[k] for k, v in state_dict_cls.items()}

# base_model.load_state_dict(uniform_soup_base_h)
# classifier = get_model_from_sd(uniform_soup_cls_h, base_model)

# base_model = base_model.to(device)
# classifier = classifier.to(device)

# val_accuracies,val_outputs, val_labels = eval_full_soup(args, val_loader,  base_model, classifier )

# val_accuracies
# results = {'model_name' : f'uniform_soup_h'}
# results['f1_sarcastic'] = val_accuracies['f1_sarcastic']
# results['f1_score'] = val_accuracies['f1_score_sarc']
# results['total_models'] = n+1
# results['all_models_name'] = all_models_name

# with open(UNIFORM_SOUP_RESULTS_FILE_H, 'a+') as f:
#     f.write(json.dumps(results) + '\n')


# uniform_soup_base_model = base_model
# uniform_soup_classifier = classifier
# test_accuracies,test_outputs, test_labels = eval_full_soup(args, test_loader,  uniform_soup_base_model, uniform_soup_classifier )

# test_accuracies



# results = {'model_name' : f'uniform_soup_t'}


# results['f1_sarcastic'] = test_accuracies['f1_sarcastic']
# results['f1_score'] = test_accuracies['f1_score_sarc']
# results['total_models'] = n+1
# results['all_models_name'] = all_models_name

# with open(UNIFORM_SOUP_RESULTS_FILE_T, 'a+') as f:
#     f.write(json.dumps(results) + '\n')

# uniform_soup_model_db_h = pd.read_json(UNIFORM_SOUP_RESULTS_FILE_H, lines=True)
# print(uniform_soup_model_db_h)

# uniform_soup_model_db_t = pd.read_json(UNIFORM_SOUP_RESULTS_FILE_T, lines=True)
# print(uniform_soup_model_db_t)

# # shutil.copy(f"uniform_soup_results_h.jsonl", f"drive/MyDrive/iSarcasm/{folder_path}/mix/results/uniform_soup_results_h.jsonl")
# # shutil.copy(f"uniform_soup_results_t.jsonl", f"drive/MyDrive/iSarcasm/{folder_path}/mix/results/uniform_soup_results_t.jsonl")


In [56]:
#truncate_arr = [False, True]
# Step 4: Greedy Soup.
#for truncate_b in truncate_arr:
truncate_b = True
INDIVIDUAL_MODEL_RESULTS_FILE_H = f"drive/MyDrive/iSarcasm/{folder_path}/mix/results/results_h.jsonl"
if (truncate_b == True):
      GREEDY_SOUP_RESULTS_FILE_H = f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/greedy_soup_results_h2.jsonl'
      GREEDY_SOUP_RESULTS_FILE_T = f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/greedy_soup_results_t2.jsonl'

else:
      GREEDY_SOUP_RESULTS_FILE_H = f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/greedy_soup_results_h.jsonl'
      GREEDY_SOUP_RESULTS_FILE_T = f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/greedy_soup_results_t.jsonl'

# if os.path.exists(GREEDY_SOUP_RESULTS_FILE_H):
#     os.remove(GREEDY_SOUP_RESULTS_FILE_H)
# if os.path.exists(GREEDY_SOUP_RESULTS_FILE_T):
#     os.remove(GREEDY_SOUP_RESULTS_FILE_T)

# print("n", n+1)

greedy_soup_params_base_h = None
greedy_soup_params_cls_h = None


individual_model_db_h = pd.read_json(INDIVIDUAL_MODEL_RESULTS_FILE_H, lines=True)
NUM_MODELS = len(individual_model_db_h)
n = NUM_MODELS -1
# Sort models by decreasing accuracy on the held-out validation set ImageNet2p
# (We call the held out-val set ImageNet2p because it is 2 percent of ImageNet train)

individual_model_val_accs_h = {}
for _, row in individual_model_db_h.iterrows():
    individual_model_val_accs_h[row['model_name']] = row['f1_sarcastic']

individual_model_val_accs_h = sorted(individual_model_val_accs_h.items(), key=operator.itemgetter(1))
individual_model_val_accs_h.reverse()
sorted_models = [x[0] for x in individual_model_val_accs_h]
# sorted_models
# Start the soup by using the first ingredient.
greedy_soup_ingredients_h = None
greedy_soup_ingredients_h = [sorted_models[0]]
best_model_val = None
best_model_val = sorted_models[0]
base_model = TransformerLayer(pretrained_path=args['pretrained_path'], both=True).to(device)

args["model_location"]
base_dict
base_dict['16_FL_1e-05_1_16826_alpha_e_3_1e-05_16.pth']
var = base_dict[sorted_models[0]+'.pth']
greedy_soup_params_base_h = torch.load(args["model_location"]+ f'/{var}')

var1 = cls_dict[sorted_models[0]+'.pth']
greedy_soup_params_cls_h = torch.load(args["model_location"]+ f'/{var1}')
# print("base",greedy_soup_params_base_h['transformer.encoder.layer.4.attention.self.query.weight'])
# print("cls",greedy_soup_params_cls_h['model.rnn_cell_list.0.h2h.weight'])
best_val_acc_so_far = None

best_val_acc_so_far = individual_model_val_accs_h[0][1] if not truncate_b else truncate(individual_model_val_accs_h[0][1],2)
print("best_val_acc_so_far", best_val_acc_so_far)
num_ingredients = len(greedy_soup_ingredients_h)
print("n_ingredients", num_ingredients)
sorted_models

best_val_acc_so_far 0.62
n_ingredients 1


['32_FL_2e-05_1_16826_alpha_e_1_2e-05_32',
 '32_FL_1e-05_1_16826_alpha_e_1_1e-05_32',
 '16_FL_2e-05_1_16826_alpha_e_2_2e-05_16',
 '32_FL_2e-05_1_16826_alpha_e_2_2e-05_32',
 '32_FL_3e-05_1_16826_alpha_e_2_3e-05_32',
 '16_FL_2e-05_1_16826_alpha_e_1_2e-05_16',
 '32_FL_4e-05_1_16826_alpha_e_2_4e-05_32',
 '32_FL_1e-05_1_16826_alpha_e_2_1e-05_32',
 '16_FL_5e-05_1_16826_alpha_e_1_5e-05_16',
 '16_FL_3e-05_1_16826_alpha_e_2_3e-05_16',
 '32_FL_3e-05_1_16826_alpha_e_1_3e-05_32',
 '32_FL_5e-05_1_16826_alpha_e_2_5e-05_32',
 '32_FL_5e-05_1_16826_alpha_e_1_5e-05_32',
 '16_FL_4e-05_1_16826_alpha_e_1_4e-05_16',
 '32_FL_4e-05_1_16826_alpha_e_1_4e-05_32',
 '32_FL_6e-05_1_16826_alpha_e_1_6e-05_32',
 '16_FL_4e-05_1_16826_alpha_e_2_4e-05_16',
 '16_FL_3e-05_1_16826_alpha_e_1_3e-05_16',
 '32_FL_6e-05_1_16826_alpha_e_2_6e-05_32',
 '16_FL_5e-05_1_16826_alpha_e_2_5e-05_16']

In [64]:
sorted_models[0]

'32_FL_2e-05_1_16826_alpha_e_1_2e-05_32'

In [ ]:
# # greedy_soup_ingredients_h
# greedy_soup_params_base_h = torch.load(f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/greedy_soup_base.pth')
# greedy_soup_params_cls_h = torch.load(f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/greedy_soup_cls.pth')

In [57]:

# best_val_acc_so_far =0.6267857142857142
# greedy_soup_ingredients_h = ['32_FL_2e-05_1_16826_alpha_e_1_2e-05_32',
#  '16_FL_2e-05_1_16826_alpha_e_2_2e-05_16',
#  '32_FL_2e-05_1_16826_alpha_e_2_2e-05_32']

In [57]:
print("-----------------------------------------------------------------------")
# Now, iterate through all models and consider adding them to the greedy soup.
# for i in range(1, NUM_MODELS):
for i in range(1, len(sorted_models)):
    print(sorted_models[i])
    # x key of dict
    # y is the file name of the base
    arr = sorted_models[i].split("_")
    # print(sorted_models[i], arr)

    # print(f'Testing model {i} of {NUM_MODELS}')

    # Get the potential greedy soup, which consists of the greedy soup with the new model added.
    var3 = base_dict[sorted_models[i]+'.pth']
    new_ingredient_params_base = torch.load(args["model_location"]+ f'/{var3}')

    var4 = cls_dict[sorted_models[i]+'.pth']
    new_ingredient_params_cls = torch.load(args["model_location"]+ f'/{var4}')


    # print("base new",new_ingredient_params_base['transformer.encoder.layer.4.attention.self.query.weight'])
    # print("cls new",new_ingredient_params_cls['model.rnn_cell_list.0.h2h.weight'])

    num_ingredients = len(greedy_soup_ingredients_h)
    print("n_ingredients", num_ingredients)


    potential_greedy_soup_params_base_h = {
                k : greedy_soup_params_base_h[k].clone() * (num_ingredients / (num_ingredients + 1.)) +
                    new_ingredient_params_base[k].clone() * (1. / (num_ingredients + 1))
                for k in new_ingredient_params_base
            }
    potential_greedy_soup_params_cls_h = {
                k : greedy_soup_params_cls_h[k].clone() * (num_ingredients / (num_ingredients + 1.)) +
                    new_ingredient_params_cls[k].clone() * (1. / (num_ingredients + 1))
                for k in new_ingredient_params_cls
            }
    # print("base new",potential_greedy_soup_params_base_h['transformer.encoder.layer.4.attention.self.query.weight'])
    # print("poten cls new",potential_greedy_soup_params_cls_h['model.rnn_cell_list.0.h2h.weight'])

    base_model.load_state_dict(potential_greedy_soup_params_base_h)
    # Run the potential greedy soup on the held-out val set.
    classifier = get_model_from_sd(potential_greedy_soup_params_cls_h, base_model)

    base_model = base_model.to(device)
    classifier = classifier.to(device)

    results = {}
    val_accuracies,val_outputs, val_labels = eval_full_soup(args, val_loader,  base_model, classifier )
    # results['accuracy'] = val_accuracies['accuracy'] if not truncate_b else truncate(val_accuracies['accuracy']  ,2)

    results['f1_score'] = val_accuracies['f1_score_sarc']
    results['f1_sarcastic'] = val_accuracies['f1_sarcastic']

    # results[f'uniform_soup'] = f1_sarcastic
    f1_score_macro = results['f1_score']
    # If accuracy on the held-out val set increases, add the new model to the greedy soup.

    # print(f'Potential greedy soup val acc {results['f1_sarcastic']}, best so far {best_val_acc_so_far}.')

    test_accuracies,test_outputs, test_labels = eval_full_soup(args, test_loader,  base_model, classifier )
    print("val_accuracies",val_accuracies)
    print("test_accuracies",test_accuracies)
    print("i", i)
    if (results['f1_sarcastic']  if not truncate_b else truncate( results['f1_sarcastic'] ,2) )>= best_val_acc_so_far:

        greedy_soup_ingredients_h.append(sorted_models[i])
        best_val_acc_so_far = results['f1_sarcastic'] if not truncate_b else truncate( results['f1_sarcastic'] ,2)
        greedy_soup_params_base_h = potential_greedy_soup_params_base_h
        greedy_soup_params_cls_h = potential_greedy_soup_params_cls_h
        print(f'Adding to soup. New soup is {greedy_soup_ingredients_h}')

        torch.save(greedy_soup_params_base_h, f"drive/MyDrive/iSarcasm/{folder_path}/mix/results/greedy_soup_base.pth")
        torch.save(greedy_soup_params_cls_h, f"drive/MyDrive/iSarcasm/{folder_path}/mix/results/greedy_soup_cls.pth")

        base_model.load_state_dict(greedy_soup_params_base_h)
        classifier = get_model_from_sd( greedy_soup_params_cls_h, base_model)
        base_model = base_model.to(device)
        classifier = classifier.to(device)




    print("-----------------------------------------------------------------------")


-----------------------------------------------------------------------
32_FL_1e-05_1_16826_alpha_e_1_1e-05_32
n_ingredients 1
2510/2510: [===============================>] - ETA 7.3s
3000/3000: [===============================>] - ETA 0.0s
val_accuracies {'acc_sarc': 0.8197983014861996, 'f1_sarcastic': 0.6208053691275168, 'f1_score_sarc': 0.7513640220768201}
test_accuracies {'acc_sarc': 0.749, 'f1_sarcastic': 0.6342884895580378, 'f1_score_sarc': 0.7216101163852102}
i 1
Adding to soup. New soup is ['32_FL_2e-05_1_16826_alpha_e_1_2e-05_32', '32_FL_1e-05_1_16826_alpha_e_1_1e-05_32']
-----------------------------------------------------------------------
16_FL_2e-05_1_16826_alpha_e_2_2e-05_16
n_ingredients 2
2510/2510: [===============================>] - ETA 0.1s
3000/3000: [===============================>] - ETA 0.0s
val_accuracies {'acc_sarc': 0.8285562632696392, 'f1_sarcastic': 0.6221441124780316, 'f1_score_sarc': 0.7556882334672487}
test_accuracies {'acc_sarc': 0.7556666666666667, '

In [58]:
greedy_soup_ingredients_h_str = ', '.join([str(elem) for i,elem in enumerate(greedy_soup_ingredients_h)])


In [59]:
# Finally, evaluate the greedy soup.
base_model.load_state_dict(greedy_soup_params_base_h)

<All keys matched successfully>

In [60]:

classifier = get_model_from_sd( greedy_soup_params_cls_h, base_model)

In [61]:

val_accuracies,val_outputs, val_labels = eval_full_soup(args, val_loader,  base_model, classifier )

val_accuracies
test_accuracies,test_outputs, test_labels = eval_full_soup(args, test_loader,  base_model, classifier )

test_accuracies

2510/2510: [===============================>] - ETA 0.1s
3000/3000: [===============================>] - ETA 0.0s


{'acc_sarc': 0.7556666666666667,
 'f1_sarcastic': 0.6358668653750621,
 'f1_score_sarc': 0.7260096804928985}

In [62]:




base_model = base_model.to(device)
classifier = classifier.to(device)
results = {'model_name' : f'greedy_soup_h'}

all_models_name = ', '.join(individual_model_db_h['model_name'])


results['f1_sarcastic'] = val_accuracies['f1_sarcastic']
results['f1_score'] = val_accuracies['f1_score_sarc']

results['num_models']=num_ingredients
results['models']= greedy_soup_ingredients_h_str
results['total_models']= n+1
results['all_models_name']= all_models_name

with open(GREEDY_SOUP_RESULTS_FILE_H, 'a+') as f:
    f.write(json.dumps(results) + '\n')

greedy_soup_base_model = base_model
greedy_soup_classifier = classifier



results = {'model_name' : f'greedy_soup_t'}
results['f1_sarcastic'] = test_accuracies['f1_sarcastic']
results['f1_score'] = test_accuracies['f1_score_sarc']
results['num_models']=num_ingredients
results['models']= greedy_soup_ingredients_h_str
results['total_models']= n+1
results['all_models_name']= all_models_name

with open(GREEDY_SOUP_RESULTS_FILE_T, 'a+') as f:
    f.write(json.dumps(results) + '\n')


# ls drive/MyDrive/iSarcasm/Tuning/
# shutil.copy(f"{GREEDY_SOUP_RESULTS_FILE_H}", f"drive/MyDrive/iSarcasm/{folder_path}/mix/results/{GREEDY_SOUP_RESULTS_FILE_H}")
# shutil.copy(f"{GREEDY_SOUP_RESULTS_FILE_T}", f"drive/MyDrive/iSarcasm/{folder_path}/mix/results/{GREEDY_SOUP_RESULTS_FILE_T}")


greedy_soup_model_db_h = pd.read_json(GREEDY_SOUP_RESULTS_FILE_H, lines=True)
print(greedy_soup_model_db_h)
# shutil.copy(f"{best_model_f}",  f"{folder_dic[data_i]}/{model_i}/results_i/best_model_f.jsonl")

greedy_soup_model_db_t = pd.read_json(GREEDY_SOUP_RESULTS_FILE_T, lines=True)
print(greedy_soup_model_db_t)

      model_name  f1_sarcastic  f1_score  num_models  \
0  greedy_soup_h      0.620921  0.751245           2   
1  greedy_soup_h      0.620921  0.751245           2   
2  greedy_soup_h      0.630397  0.759787           2   
3  greedy_soup_h      0.626840  0.757663           4   
4  greedy_soup_h      0.620330  0.753691           7   
5  greedy_soup_h      0.623468  0.756293           6   

                                              models  total_models  \
0  16_FL_1e-05_1_16826_alpha_e_2_1e-05_16, 16_FL_...             6   
1  16_FL_1e-05_1_16826_alpha_e_2_1e-05_16, 16_FL_...             8   
2  16_FL_1e-05_1_16826_alpha_e_2_1e-05_16, 32_FL_...            12   
3  16_FL_1e-05_1_16826_alpha_e_2_1e-05_16, 32_FL_...            16   
4  16_FL_1e-05_1_16826_alpha_e_2_1e-05_16, 32_FL_...            16   
5  32_FL_2e-05_1_16826_alpha_e_1_2e-05_32, 32_FL_...            20   

                                     all_models_name  
0  16_FL_1e-05_1_16826_alpha_e_1_1e-05_16, 16_FL_...  
1  16_

In [ ]:
# GREEDY_SOUP_RESULTS_FILE_H = 'greedy_soup_results_h2.jsonl'
# GREEDY_SOUP_RESULTS_FILE_T = 'greedy_soup_results_t2.jsonl'

In [ ]:
greedy_soup_model_db_h = pd.read_json(f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/greedy_soup_results_h2.jsonl', lines=True)
print(greedy_soup_model_db_h)

greedy_soup_model_db_t = pd.read_json(f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/greedy_soup_results_t2.jsonl', lines=True)
print(greedy_soup_model_db_t)

In [ ]:
greedy_soup_model_db_t